<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 使用 Python 呼叫 API

OpenAI 官方提供有 openai 套件, 可以簡化直接使用 requests 模組的複雜度。

## 2-1 使用官方 openai 套件

### 安裝與使用 openai **套件**

In [ ]:
!pip install openai

In [ ]:
import openai
client = openai.OpenAI(api_key='你的金鑰')

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你好"}
    ]
)

檢視傳回物件

In [ ]:
print(reply)

In [ ]:
!pip install rich

In [ ]:
from rich import print as pprint
pprint(reply)

In [ ]:
print(reply.choices[0].message.content)

### 直接透過模組使用 API

In [ ]:
openai.api_key = '你的金鑰'

reply = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你好"}
    ]
)

print(reply.choices[0].message.content)

In [ ]:
pprint(reply)

In [ ]:
print(reply.choices[0].message.content)

### 傳遞多筆訊息

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system",
         "content":"你是條住在深海、只會台灣中文的魚"},
        {"role":"user", "content": "你住的地方很亮嗎？"}
    ]
)

In [ ]:
print(reply.choices[0].message.content)

### 設定與隱藏金鑰的方法

Colab 提供有 Secret, 可以依帳戶儲存機密資料, 並設定是否允許必記本存取。機密資料不會隨筆記本分享出去, 只有帳戶擁有者才能讀取。

In [ ]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

程式中就可以不用出現金鑰了

## 2-2 認識 token

### 使用 tiktoken 套件計算精確 token 數


In [ ]:
!pip install tiktoken
import tiktoken

In [ ]:
encoder = tiktoken.encoding_for_model('gpt-3.5-turbo')
print(encoder.name)
encoder = tiktoken.encoding_for_model('gpt-4')
print(encoder.name)

In [ ]:
tokens = encoder.encode("你好")
print(tokens)
print(encoder.decode(tokens))

### ChatML 標記語言

In [ ]:
print(encoder.encode("user"))
print(encoder.encode("assistant"))
print(encoder.encode("system"))
print(encoder.encode("\n"))

計算 message 總 tokens 數

In [ ]:
def tokens_in_messages(messages, model="gpt-3.5-turbo",
                       show=False):
    totals = 0
    for message in messages:
        for k in message:
            if k == "content":
                totals += 4 # <|im_start|>user\n{內容}<|im_end|>
                totals += len(encoder.encode(message[k]))
    totals += 3 # <|im_start|>assistant<|message|>
    return totals

In [ ]:
print(tokens_in_messages([
        {"role":"user", "content": "你好"}
    ]))

## 2-3 使用 Python requests 模組呼叫 API

### 使用 Python requests 模組

Python 提供有 requests 模組, 可以快速發送 HTTP 請求。<br>
利用 requests 模組叫用 API。

In [ ]:
# 如果是要在本機上執行, 必須安裝 requests 套件
!pip install requests

In [ ]:
import requests
import os

response = requests.post(
    'https://api.openai.com/v1/chat/completions',
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {userdata.get("OPENAI_API_KEY")}'
    },
    json = {
        'model': 'gpt-3.5-turbo',
        "messages": [{"role": "user", "content": "你好"}]}
)

In [ ]:
reply = response.json()

In [ ]:
pprint(reply)

In [ ]:
print(reply["choices"][0]["message"]["content"])

### 利用 curl 工具快速測試 API


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [ ]:
%%bash
curl -s https://api.openai.com/v1/chat/completions \
-H 'Content-Type: application/json' \
-H "Authorization: Bearer $OPENAI_API_KEY" \
-d '{
      "model": "gpt-3.5-turbo",
      "messages": [
        {
          "role": "user", "content": "你好"
        }
      ]
    }'

## 2-4 加入組織成員

In [ ]:
client = openai.OpenAI(
    api_key=userdata.get('OPENAI_API_KEY'),
    organization='你的組織識別碼'
)

reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你好"}
    ]
)

print(reply.choices[0].message.content)

In [ ]:
import requests
import os

response = requests.post(
    'https://api.openai.com/v1/chat/completions',
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {userdata.get("OPENAI_API_KEY")}',
        'OpenAI-Organization': '你的組織識別碼'
    },
    json = {
        'model': 'gpt-3.5-turbo',
        "messages": [{"role": "user", "content": "你好"}]}
)

print(response.json()['choices'][0]['message']['content'])

In [ ]:
%%bash
curl -s https://api.openai.com/v1/chat/completions \
-H 'Content-Type: application/json' \
-H "Authorization: Bearer $OPENAI_API_KEY" \
-H "OpenAI-Organization: 請換成你的組織識別碼" \
-d '{
      "model": "gpt-3.5-turbo",
      "messages": [
        {
          "role": "user", "content": "你好"
        }
      ]
    }'